##### Copyright 2018 The TensorFlow Authors.

In [0]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# TensorBoard Quickstart

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/not_a_real_link"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />Currently N/A</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/tensorboard/blob/master/docs/r2/quickstart.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/tensorboard/blob/master/docs/r2/quickstart.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
</table>

In machine learning, to improve something you often need to be able to measure it. TensorBoard is a tool for providing the measurements and visualizations needed during the machine learning workflow. It enables tracking experiment metrics like loss and accuracy, visualizing the model graph, projecting embeddings to a lower dimensional space, and much more.

This quickstart will show how to quickly get started with TensorBoard. The remaining guides in this website provide more details on specific capabilities, many of which are not included here. 

In [0]:
!pip install tf-nightly-2.0-preview
# Load the TensorBoard notebook extension
%load_ext tensorboard.notebook 

In [0]:
import tensorflow as tf
from datetime import datetime

In [0]:
# Clear any logs from previous runs
!rm -rf ./logs/ 

Using the [FashionMNIST](https://github.com/zalandoresearch/fashion-mnist) dataset as the example, normalize the data and write a function that creates a simple Keras model for classifying the images into 10 classes.

In [0]:
mnist = tf.keras.datasets.mnist

(x_train, y_train),(x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

def create_model():
  return tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(10, activation='softmax')
  ])

## Using TensorBoard with Keras Model.fit()

When training with Keras's [Model.fit()](https://www.tensorflow.org/api_docs/python/tf/keras/models/Model#fit), adding the [TensorBoard callback](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/callbacks/TensorBoard) will ensure logs are created and stored. Additionally, enable histogram computation every epoch with `histogram_freq=1` (this is off by default)

Place the logs in a timestamped subdirectory to allow easy selection of different training runs.

In [0]:
model = create_model()
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

log_dir="logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

model.fit(x=x_train, 
          y=y_train, 
          epochs=5, 
          validation_data=(x_test, y_test), 
          callbacks=[tensorboard_callback])

Start TensorBoard through the command line or within a notebook experience. The two interfaces are generally the same. In notebooks, use the `%tensorboard` line magic. On the command line, run the same command without "%".

In [0]:
%tensorboard --logdir logs/fit

<!-- <img class="tfo-display-only-on-site" src="images/quickstart_model_fit.png"/> -->

A brief overview of the plugins shown (tabs in top navigation bar):

* The **Scalars Plugin** shows how the loss and metrics change with every epoch. You can use it to also track training speed, learning rate, and other scalar values
* The **Graphs Plugin** helps you visualize your model. In this case, the Keras graph of layers is shown which can help you ensure it is built correctly. 
* The **Distributions Plugin** and the **Histograms Plugin** show the distribution of a Tensor over time. This can be useful to visualize weights and biases and verify that they are changing in an expected way.

Additional TensorBoard plugins are automatically enabled when you log other types of data. For example, the Keras TensorBoard callback lets you log images and embeddings as well. You can see what other plugins are available in TensorBoard by clicking on the "inactive" dropdown towards the top right.

## Using TensorBoard with other methods

When training with methods such as with [`tf.GradientTape()`](https://www.tensorflow.org/api_docs/python/tf/GradientTape), use `tf.summary` to log the required information.

Use the same dataset as above, but convert it to `tf.data.Dataset` to take advantage of batching capabilities:

In [0]:
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test))

train_dataset = train_dataset.shuffle(60000).batch(128)
test_dataset = test_dataset.batch(128)

The training code is based on the metrics section of the [Effective TensorFlow 2.0](https://github.com/tensorflow/docs/blob/master/site/en/r2/guide/effective_tf2.md) guide:

In [0]:
compute_loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optimizer = tf.keras.optimizers.Adam()
compute_accuracy = tf.keras.metrics.SparseCategoricalAccuracy()

def train_step(model, optimizer, x_train, y_train):
  with tf.GradientTape() as tape:
    logits = model(x_train, training=True)
    loss = compute_loss(y_train, logits)
    compute_accuracy(y_train, logits)
  grads = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(grads, model.trainable_variables))
  return loss

Note that `compute_accuracy` is a stateful metric that you can use to accumulate every train step and log later, as shown in the next cell. 

Define the training and test functions. Use `tf.summary.scalar()` to log our metrics (loss and accuracy). You have control over which metrics to log and how often you do it. Other `tf.summary` functions enable logging other types of data.

In [0]:
def train(model, optimizer, dataset, log_freq=50):
  avg_loss = tf.keras.metrics.Mean('loss', dtype=tf.float32)

  for x_train, y_train in dataset:
    loss = train_step(model, optimizer, x_train, y_train)
    avg_loss(loss)

    if tf.equal(optimizer.iterations % log_freq, 0):
      tf.summary.scalar('loss', avg_loss.result(), step=optimizer.iterations)
      tf.summary.scalar('accuracy', compute_accuracy.result(), step=optimizer.iterations)
      print('Step #%d\tLoss: %.6f' % (optimizer.iterations, loss))
    avg_loss.reset_states()
    compute_accuracy.reset_states()

def test(model, dataset, step_num):
  avg_loss = tf.keras.metrics.Mean('loss', dtype=tf.float32)

  for (x_test, y_test) in dataset:
    logits = model(x_test, training=False)
    avg_loss(compute_loss(y_test, logits))
    compute_accuracy(y_test, logits)
  print('Model test set loss: {:0.4f} accuracy: {:0.2f}%'.format(
      avg_loss.result(), compute_accuracy.result() * 100))
  tf.summary.scalar('loss', avg_loss.result(), step=step_num)
  tf.summary.scalar('accuracy', compute_accuracy.result(), step=step_num)
    

Train and test the model within the scope of summary writers to write the summaries to disk.

In [0]:
current_time = datetime.now().strftime("%Y%m%d-%H%M%S")
train_log_dir = 'logs/gradient_tape/' + current_time + '/train'
test_log_dir = 'logs/gradient_tape/' + current_time + '/test'
train_summary_writer = tf.summary.create_file_writer(train_log_dir)
test_summary_writer = tf.summary.create_file_writer(test_log_dir)

model = create_model() # reset our model

with train_summary_writer.as_default():
  train(model, optimizer, train_dataset)

with test_summary_writer.as_default():
  test(model, test_dataset, optimizer.iterations)

Open TensorBoard in the same way way but point it at a different log directory. After starting TensorBoard, you can rerun the previous cells to start training again and monitor the training within TensorBoard while it's happening.

Unlike the experiments with `Model.fit()`, in this case the test set is only used at the end of training. You could adapt the above steps to do multiple epochs and evaluate on the test set at the end of every epoch.

In [0]:
%tensorboard --logdir logs/gradient_tape

<!-- <img class="tfo-display-only-on-site" src="images/quickstart_gradient_tape.png"/> -->

That's it! You have now seen how to use TensorBoard both through the Keras callback and through `tf.summary` for more custom scenarios. 